Imports

In [ ]:
%pip install numpy
%pip install pandas
%pip install sklearn
%pip install matplotlib
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from scipy.io import arff
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import DBSCAN, KMeans
import itertools as it
from matplotlib import pyplot as plt


pd.set_option('display.max_rows', 500)

Load data

In [ ]:
columns = ["net profit / total assets", "total liabilities / total assets", "working capital / total assets", "current assets / short-term liabilities", "[(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365", "retained earnings / total assets", "EBIT / total assets", "book value of equity / total liabilities", "sales / total assets", "equity / total assets", "(gross profit + extraordinary items + financial expenses) / total assets", "gross profit / short-term liabilities", "(gross profit + depreciation) / sales", "(gross profit + interest) / total assets", "(total liabilities * 365) / (gross profit + depreciation)", "(gross profit + depreciation) / total liabilities", "total assets / total liabilities", "gross profit / total assets", "gross profit / sales", "(inventory * 365) / sales", "sales (n) / sales (n-1)", "profit on operating activities / total assets", "net profit / sales", "gross profit (in 3 years) / total assets", "(equity - share capital) / total assets", "(net profit + depreciation) / total liabilities", "profit on operating activities / financial expenses", "working capital / fixed assets", "logarithm of total assets", "(total liabilities - cash) / sales", "(gross profit + interest) / sales", "(current liabilities * 365) / cost of products sold", "operating expenses / short-term liabilities", "operating expenses / total liabilities", "profit on sales / total assets", "total sales / total assets", "(current assets - inventories) / long-term liabilities", "constant capital / total assets", "profit on sales / sales", "(current assets - inventory - receivables) / short-term liabilities", "total liabilities / ((profit on operating activities + depreciation) * (12/365))", "profit on operating activities / sales", "rotation receivables + inventory turnover in days", "(receivables * 365) / sales", "net profit / inventory", "(current assets - inventory) / short-term liabilities", "(inventory * 365) / cost of products sold", "EBITDA (profit on operating activities - depreciation) / total assets", "EBITDA (profit on operating activities - depreciation) / sales", "current assets / total liabilities", "short-term liabilities / total assets", "(short-term liabilities * 365) / cost of products sold)", "equity / fixed assets", "constant capital / fixed assets", "working capital", "(sales - cost of products sold) / sales", "(current assets - inventory - short-term liabilities) / (sales - gross profit - depreciation)", "total costs /total sales", "long-term liabilities / equity", "sales / inventory", "sales / receivables", "(short-term liabilities *365) / sales", "sales / short-term liabilities", "sales / fixed assets", "IsGoingToBankrupt"]
data_1_year = arff.loadarff('datasets/1year.arff')
data_2_year = arff.loadarff('datasets/2year.arff')
data_3_year = arff.loadarff('datasets/3year.arff')
data_4_year = arff.loadarff('datasets/4year.arff')
data_5_year = arff.loadarff('datasets/5year.arff')

df_1_year = pd.DataFrame(data_1_year[0])
df_1_year.set_axis(columns, inplace=True, axis=1)
df_1_year['IsGoingToBankrupt'] = df_1_year.apply(lambda x: x[64] == b'1', axis=1)

df_2_year = pd.DataFrame(data_2_year[0])
df_2_year.set_axis(columns, inplace=True, axis=1)
df_2_year['IsGoingToBankrupt'] = df_2_year.apply(lambda x: x[64] == b'1', axis=1)

df_3_year = pd.DataFrame(data_3_year[0])
df_3_year.set_axis(columns, inplace=True, axis=1)
df_3_year['IsGoingToBankrupt'] = df_3_year.apply(lambda x: x[64] == b'1', axis=1)

df_4_year = pd.DataFrame(data_4_year[0])
df_4_year.set_axis(columns, inplace=True, axis=1)
df_4_year['IsGoingToBankrupt'] = df_4_year.apply(lambda x: x[64] == b'1', axis=1)

df_5_year = pd.DataFrame(data_5_year[0])
df_5_year.set_axis(columns, inplace=True, axis=1)
df_5_year['IsGoingToBankrupt'] = df_5_year.apply(lambda x: x[64] == b'1', axis=1)

Choose columns

In [ ]:
columns_left = ['net profit / total assets', 'total liabilities / total assets',
       'sales / total assets', 'sales (n) / sales (n-1)',
       'working capital / fixed assets', 'sales / fixed assets',
       'IsGoingToBankrupt']
df_1_year = df_1_year[columns_left]
df_2_year = df_2_year[columns_left]
df_3_year = df_3_year[columns_left]
df_4_year = df_4_year[columns_left]
df_5_year = df_5_year[columns_left]

Drop null values

In [ ]:
df_1_year.dropna(inplace=True)
df_2_year.dropna(inplace=True)
df_3_year.dropna(inplace=True)
df_4_year.dropna(inplace=True)
df_5_year.dropna(inplace=True)

Calculate covariance matrix

In [ ]:
def calc_corr(data_frame):
    columns = data_frame.columns
    results = pd.DataFrame(columns=columns, index=columns)
    for column1 in columns:
        for column2 in columns:
            if column1 == column2:
                continue
            corr, _ = spearmanr(data_frame[column1], data_frame[column2])
            results[column1][column2] = corr
    return results

def eliminate_high_corr_columns(data_frame_corr_mat, eps = 0.9, verbose = True):
    columns = data_frame_corr_mat.columns
    column1 = 0
    while column1 < len(columns):
        to_drop = []
        for column2 in range(column1 + 1, len(columns)):
            if column1 == column2:
                continue
            corr = data_frame_corr_mat[columns[column1]][columns[column2]]
            if abs(corr) > eps:
                if verbose:
                    print(f'Corr: {corr}, DELETING {columns[column2]}')
                to_drop.append(columns[column2])
        columns = columns.drop(to_drop)
        column1 += 1
    return columns

In [ ]:
# columns_after_drop = eliminate_high_corr_columns(calc_corr(df_1_year), 0.5, verbose=False)
# columns_after_drop = columns_after_drop.drop(['(inventory * 365) / sales' ,
                                                # '(current assets - inventory - receivables) / short-term liabilities',
                                                #  '(current assets - inventories) / long-term liabilities',
                                                #  '(current liabilities * 365) / cost of products sold',
                                                #  '(receivables * 365) / sales',
                                                #  'logarithm of total assets',
                                                #  'retained earnings / total assets'])
display(calc_corr(df_1_year))
display(len(calc_corr(df_1_year)))

Separate bankrupt and non-bankrupt

In [ ]:
year_1_b = df_1_year.query('IsGoingToBankrupt == True').drop(columns=['IsGoingToBankrupt'])
year_1_nb = df_1_year.query('IsGoingToBankrupt == False').drop(columns=['IsGoingToBankrupt'])

year_1_b = year_1_b.reset_index().drop(columns=['index'])
year_1_nb = year_1_nb.reset_index().drop(columns=['index'])


year_2_b = df_2_year.query('IsGoingToBankrupt == True').drop(columns=['IsGoingToBankrupt'])
year_2_nb = df_2_year.query('IsGoingToBankrupt == False').drop(columns=['IsGoingToBankrupt'])

year_2_b = year_2_b.reset_index().drop(columns=['index'])
year_2_nb = year_2_nb.reset_index().drop(columns=['index'])


year_3_b = df_3_year.query('IsGoingToBankrupt == True').drop(columns=['IsGoingToBankrupt'])
year_3_nb = df_3_year.query('IsGoingToBankrupt == False').drop(columns=['IsGoingToBankrupt'])

year_3_b = year_3_b.reset_index().drop(columns=['index'])
year_3_nb = year_3_nb.reset_index().drop(columns=['index'])


year_4_b = df_4_year.query('IsGoingToBankrupt == True').drop(columns=['IsGoingToBankrupt'])
year_4_nb = df_4_year.query('IsGoingToBankrupt == False').drop(columns=['IsGoingToBankrupt'])

year_4_b = year_4_b.reset_index().drop(columns=['index'])
year_4_nb = year_4_nb.reset_index().drop(columns=['index'])


year_5_b = df_5_year.query('IsGoingToBankrupt == True').drop(columns=['IsGoingToBankrupt'])
year_5_nb = df_5_year.query('IsGoingToBankrupt == False').drop(columns=['IsGoingToBankrupt'])

year_5_b = year_5_b.reset_index().drop(columns=['index'])
year_5_nb = year_5_nb.reset_index().drop(columns=['index'])

In [ ]:
print(f'In 1 year\n Num of bankrupt: {len(year_1_b)}\tNum of non-bankrupt: {len(year_1_nb)}')
print(f'In 2 year\n Num of bankrupt: {len(year_2_b)}\tNum of non-bankrupt: {len(year_2_nb)}')
print(f'In 3 year\n Num of bankrupt: {len(year_3_b)}\tNum of non-bankrupt: {len(year_3_nb)}')
print(f'In 4 year\n Num of bankrupt: {len(year_4_b)}\tNum of non-bankrupt: {len(year_4_nb)}')
print(f'In 5 year\n Num of bankrupt: {len(year_5_b)}\tNum of non-bankrupt: {len(year_5_nb)}')

Outliers

In [ ]:
def find_outliers(data_frame):
    clf = LocalOutlierFactor(n_neighbors=3)
    outliers = clf.fit_predict(data_frame)
    outliers_indexes = np.where(outliers == - 1)[0]
    rest_indexes = np.where(outliers == 1)[0]
    outliers = data_frame.loc[outliers_indexes]
    df_without_outliers = data_frame.loc[rest_indexes]
    df_without_outliers = df_without_outliers.reset_index().drop(columns=['index'])
    return outliers, df_without_outliers

In [ ]:
outliers_year_1_b, year_1_b_clean = find_outliers(year_1_b)
outliers_year_1_nb, year_1_nb_clean = find_outliers(year_1_nb)

In [ ]:
display(year_1_b_clean.describe())
display(year_1_nb_clean.describe())

In [ ]:
display(year_1_b_clean.describe())
display(outliers_year_1_b)

In [ ]:
display(year_1_nb_clean.describe())
display(outliers_year_1_nb)

Clusterization - dbscan

In [ ]:
def clusterize(data_frame, eps):
    normalized_df=(data_frame-data_frame.min())/(data_frame.max()-data_frame.min())
    clustering = DBSCAN(eps=eps).fit(normalized_df)
    output = []
    for label in range(clustering.n_features_in_):
        tmp_indexes = np.where(clustering.labels_ == label)[0]
        tmp = data_frame.loc[tmp_indexes].reset_index().drop(columns=['index'])
        output.append(tmp)
    return output

def display_clusters(clusters):
    for i, cluster in enumerate(clusters):
        if(len(cluster)):
            display(f'Cluster {i}:')
            display(f'Number of elements: {len(cluster)}')
            display(cluster.describe())

In [ ]:
display(year_1_b_clean.describe())
clusters = clusterize(year_1_b_clean, 0.1)
display_clusters(clusters)

In [ ]:
display(year_1_nb_clean.describe())
clusters = clusterize(year_1_nb_clean, 0.01)
display_clusters(clusters)

Clusterization - kmeans

In [ ]:
def clusterize_kmeans(data_frame, cluster_am):
    normalized_df=(data_frame-data_frame.min())/(data_frame.max()-data_frame.min())
    clustering = KMeans(n_clusters=cluster_am).fit(normalized_df)
    output = []
    for label in range(clustering.n_features_in_):
        tmp_indexes = np.where(clustering.labels_ == label)[0]
        tmp = data_frame.loc[tmp_indexes].reset_index().drop(columns=['index'])
        output.append(tmp)
    return output


In [ ]:
display(year_1_b_clean.describe())
clusters = clusterize_kmeans(year_1_b_clean, 4)
display_clusters(clusters)

In [ ]:
display(year_1_nb_clean.describe())
clusters = clusterize_kmeans(year_1_nb_clean, 4)
display_clusters(clusters)

Graphs - relations between atributes

In [ ]:
for A, B in it.combinations(year_1_b_clean.columns, 2):
    plt.xlabel(A)
    plt.ylabel(B)
    plt.plot(year_1_nb_clean[A], year_1_nb_clean[B], '+', label="Non-Bankrupt")
    plt.plot(year_1_b_clean[A], year_1_b_clean[B], 'x', label="Bankrupt")
    plt.legend()
    plt.show()